# **Setup and Load Data**

# Install dependecies and setup
Tensorflow for pipelining, opencv for cleaning dataset, matplotlib for visualization

In [2]:
%pip install tensorflow tensorflow-gpu opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Validation

In [4]:
%pip list

Package                      Version
---------------------------- -----------
absl-py                      1.2.0
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
asttokens                    2.0.8
astunparse                   1.6.3
attrs                        22.1.0
backcall                     0.2.0
beautifulsoup4               4.11.1
bleach                       5.0.1
cachetools                   5.2.0
certifi                      2022.6.15.1
cffi                         1.15.1
charset-normalizer           2.1.1
colorama                     0.4.5
cycler                       0.11.0
debugpy                      1.6.3
decorator                    5.1.1
defusedxml                   0.7.1
entrypoints                  0.4
executing                    1.0.0
fastjsonschema               2.16.1
flatbuffers                  2.0.7
fonttools                    4.37.1
gast                         0.4.0
google-auth                  2.11.0
google-auth-oauthlib         0.4.6

You should consider upgrading via the 'D:\project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Importing dependecies

In [5]:
import os
import tensorflow as tf
from tensorflow import keras
from keras import layers
import cv2
import imghdr
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from string import digits
import random

In [6]:
os.chdir('D:\\project')
data_dir = 'trainData'

In [7]:
os.listdir(data_dir)

['mold', 'over_ripe', 'ripe', 'rotten', 'unripe']

Sorting, Pipelining and Loading Data

In [8]:
data = tf.keras.utils.image_dataset_from_directory('trainData')

Found 4950 files belonging to 5 classes.


In [9]:
IMAGE_SIZE=224
BATCH_SIZE=64

#pre=processing
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
    )

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.1
)

train_datagen=train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

test_datagen=test_datagen.flow_from_directory(
    data_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 4457 images belonging to 5 classes.
Found 493 images belonging to 5 classes.


In [10]:
label_dict = {'mold':0, 'over_ripe':1, 'ripe':2, 'rotten':3, 'unripe':4}


In [11]:
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [12]:
model=tf.keras.Sequential()

model.add(Conv2D(64, (2,3), 5, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D())


model.add(Conv2D(32, (2,3), 5, activation='relu'))
model.add(MaxPooling2D())


model.add(Conv2D(32, (2,3), 5, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

Training the Model

In [14]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train_datagen, epochs=5, validation_data=test_datagen)

Epoch 1/5


In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()